In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2 as pg
import pandas.io.sql as psql
import geopandas as gpd
# import folium
# from folium import FeatureGroup, LayerControl, Map, Marker
import pymongo
import numpy as np

In [2]:
#connect with postgresql and extract data (4 tables)
connection = pg.connect("host=localhost dbname=project2 user=postgres password=ynot8888")
export_df = psql.read_sql('SELECT * FROM export', connection)

In [3]:
export_df.head()

,id,countries,iso_a3,y_1987,y_1988,y_1989,y_1990,y_1991,y_1992,y_1993,...,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019
0,1,China ...,CHN,1350.18,1462.10,1485.17,1606.50,1645.35,2518.08,2910.38,...,71485.7,80623.0,79164.3,99985.80,85070.40,87830.1,110030.0,123604.0,153515.0,167625.0
1,2,Japan ...,JPN,12133.30,13672.80,14617.30,16599.50,17284.30,18157.40,19059.20,...,48950.4,53183.3,48540.5,50972.10,46443.70,37974.2,44455.8,51256.5,61675.5,56161.0
2,3,Republic of Korea ...,KOR,1877.40,2313.14,2829.30,3412.02,3594.09,4328.70,5242.38,...,24583.2,24061.3,21091.6,22667.70,20613.40,19970.7,22870.6,23713.2,27838.6,27647.1
3,4,United States ...,USA,6081.37,6128.28,7081.44,7782.91,7097.02,7067.14,7471.26,...,14765.8,15577.3,15054.7,17228.10,20528.40,21896.2,20694.8,21354.7,24768.6,27403.9
4,5,United Kingdom ...,GBR,2987.15,3020.32,3305.38,3494.44,3801.41,4167.04,4732.69,...,11099.9,12422.8,10161.4,8392.47,9095.18,12641.4,12753.8,11758.7,13450.2,20950.5


In [4]:
exfiltered_df = export_df.dropna(how='any')
exfiltered_df.isnull().values.any()

False

In [5]:
# duplicateRowsDF = exfiltered_df.duplicated()
# print(duplicateRowsDF) #  duplicates no found 

In [6]:
exfiltered_df.head()

,id,countries,iso_a3,y_1987,y_1988,y_1989,y_1990,y_1991,y_1992,y_1993,...,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019
0,1,China ...,CHN,1350.18,1462.10,1485.17,1606.50,1645.35,2518.08,2910.38,...,71485.7,80623.0,79164.3,99985.80,85070.40,87830.1,110030.0,123604.0,153515.0,167625.0
1,2,Japan ...,JPN,12133.30,13672.80,14617.30,16599.50,17284.30,18157.40,19059.20,...,48950.4,53183.3,48540.5,50972.10,46443.70,37974.2,44455.8,51256.5,61675.5,56161.0
2,3,Republic of Korea ...,KOR,1877.40,2313.14,2829.30,3412.02,3594.09,4328.70,5242.38,...,24583.2,24061.3,21091.6,22667.70,20613.40,19970.7,22870.6,23713.2,27838.6,27647.1
3,4,United States ...,USA,6081.37,6128.28,7081.44,7782.91,7097.02,7067.14,7471.26,...,14765.8,15577.3,15054.7,17228.10,20528.40,21896.2,20694.8,21354.7,24768.6,27403.9
4,5,United Kingdom ...,GBR,2987.15,3020.32,3305.38,3494.44,3801.41,4167.04,4732.69,...,11099.9,12422.8,10161.4,8392.47,9095.18,12641.4,12753.8,11758.7,13450.2,20950.5


In [7]:
import_df = psql.read_sql('SELECT * FROM import', connection)
#import_df.head()

In [8]:
imfiltered_df = import_df.dropna(how='any')
imfiltered_df.isnull().values.any()
imfiltered_df.head()

,id,countries,iso_a3,y_1987,y_1988,y_1989,y_1990,y_1991,y_1992,y_1993,...,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019
0,1,China ...,CHN,834.271,996.650,1355.000,1456.050,1864.600,2549.47,3225.46,...,40923.6,43919.3,46442.4,49570.6,54460.0,64377.2,62323.2,67448.5,78303.1,82841.7
1,2,United States ...,USA,10820.300,12122.000,15206.500,15983.200,15816.700,16390.60,17301.40,...,36378.3,40711.6,43310.1,41003.5,43391.8,49372.3,44953.6,47331.9,50864.9,55532.5
2,3,Japan ...,JPN,8605.890,9601.500,11768.900,10641.200,10018.200,11377.10,13264.00,...,20573.5,20078.8,21420.5,21608.6,20095.8,23027.2,22823.7,24813.0,26987.6,26357.2
3,4,Germany ...,DEU,3130.720,3215.260,3739.110,3828.420,3584.860,3706.09,4042.04,...,12640.0,12677.9,13402.8,14132.7,14742.8,15569.8,16471.8,16991.9,18771.6,18183.6
4,5,Thailand ...,THA,509.124,596.471,822.311,852.804,887.411,1054.84,1133.79,...,13265.7,11066.0,13085.0,14287.4,13266.0,15804.6,16878.4,17213.3,17913.7,17385.4


In [9]:
# duplicateRowsDF = imfiltered_df.duplicated()
# print(duplicateRowsDF) 

In [10]:
importtop25_df = psql.read_sql('SELECT * FROM top25import', connection)
#importtop25_df.head(30)

In [11]:
im25filtered_df = importtop25_df.dropna(how='any')
im25filtered_df.isnull().values.any()
im25filtered_df.head()

,id,commodities,y_2015,y_2016,y_2017,y_2018,y_2019
0,1,Personal travel (excl education) services ...,36426.000,37731.000,42684.000,46321.000,33288.000
1,2,Refined petroleum ...,16227.992,17389.114,21687.610,25069.399,21720.734
2,3,Passenger motor vehicles ...,21541.778,21781.807,23299.388,21573.858,19092.510
3,4,Telecom equipment & parts ...,11781.035,11969.132,13412.081,14590.189,15230.420
4,5,Computers ...,7819.856,7853.593,8836.896,9763.131,10398.009


In [12]:
# duplicateRowsDF = im25filtered_df.duplicated()
# print(duplicateRowsDF) 

In [13]:
exporttop25_df = psql.read_sql('SELECT * FROM top25export', connection)
#exporttop25_df.head(30)

In [14]:
ex25filtered_df = exporttop25_df.dropna(how='any')
ex25filtered_df.isnull().values.any()
ex25filtered_df.head()

,id,commodities,y_2015,y_2016,y_2017,y_2018,y_2019
0,1,Iron ores & concentrates ...,47798.926,62617.201,61392.008,77519.590,102863.856
1,2,Coal ...,34541.191,54236.174,60378.810,69595.465,54620.426
2,3,Natural gas ...,16575.582,22307.902,30906.613,49727.233,47524.587
3,4,Education-related travel services (c) ...,24145.000,28093.000,32602.000,37824.000,39661.000
4,5,Gold ...,16585.176,18978.819,19292.604,18866.943,24393.843


In [14]:
#read in geojson file and convert into df
fname = "csv&geojson/countries.geojson"

gjs_df = gpd.read_file(fname)
gjs_df.head()

,ADMIN,ISO_A3,geometry
0,Aruba,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


In [15]:
gjs_df.insert(loc=1, column='year', value=['2019' for i in range(gjs_df.shape[0])])

In [16]:
gjs_df.insert(loc=2, column='value', value=[8888 for i in range(gjs_df.shape[0])])
gjs_df.head()

,ADMIN,year,value,ISO_A3,geometry
0,Aruba,2019,8888,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,2019,8888,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,2019,8888,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,2019,8888,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,2019,8888,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


In [17]:
print(gjs_df.shape[0])

255


In [19]:
print(imfiltered_df.iloc[0, 35])

82841.7


In [20]:
imgjs_df = gjs_df.copy()
exgjs_df = gjs_df.copy()

In [28]:
# print(imgjs_df.iloc[:, 2])

In [127]:
# countries = imfiltered_df[["countries" ]].copy()
# countries.head()

In [128]:
print(imgjs_df.iloc[0, 0])
print(imgjs_df.iloc[0, 2])

Aruba
8888


In [22]:
print(imfiltered_df.iloc[0, 1])
print(imfiltered_df.iloc[0, 2])

China                                             
CHN       


In [130]:
print(imfiltered_df.iloc[0, 34])

82841.71216


In [25]:
print(imgjs_df.iloc[0, 3])

ABW


In [32]:
print(imfiltered_df.iloc[5, 2])

GBR       


In [54]:
print([x for x in imgjs_df.iloc[:, 1]])

['2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019',

In [40]:
print([x for x in imfiltered_df.iloc[:, 2]])

['CHN       ', 'USA       ', 'JPN       ', 'DEU       ', 'THA       ', 'GBR       ', 'SGP       ', 'NZL       ', 'KOR       ', 'MYS       ', 'ITA       ', 'IDN       ', 'FRA       ', 'VNM       ', 'IND       ', 'TWN       ', 'ARE       ', 'CAN       ', 'PNG       ', 'CHE       ', 'NLD       ', 'HKG       ', 'IRL       ', 'MEX       ', 'ESP       ', 'SWE       ', 'BEL       ', 'AUT       ', 'PHL       ', 'ZAF       ', 'FJI       ', 'DNK       ', 'TUR       ', 'FIN       ', 'POL       ', 'BGD       ', 'BRN       ', 'ISR       ', 'BRA       ', 'GRC       ', 'NOR       ', 'DZA       ', 'ARG       ', 'LBY       ', 'CHL       ', 'CZE       ', 'HUN       ', 'LKA       ', 'SAU       ', 'NGA       ', 'RUS       ', 'PAK       ', 'KHM       ', 'SVK       ', 'PRT       ', 'PER       ', 'LBN       ', 'PRI       ', 'QAT       ', 'EST       ', 'BMU       ', 'CYP       ', 'VUT       ', 'ROU       ', 'EGY       ', 'COL       ', 'LUX       ', 'MUS       ', 'KWT       ', 'SLB       ', 'SVN       ', 'COD 

In [53]:
#Populate imgjs_df with 2019 import values
#remove the blank space from iso_a3 column

for k in range(imfiltered_df.shape[0]):
        imfiltered_df.iloc[k, 2] = imfiltered_df.iloc[k, 2].strip()
        imfiltered_df.iloc[k, 1] = imfiltered_df.iloc[k, 1].strip()
        
for i in range(imgjs_df.shape[0]):
    imgjs_df.iloc[i, 2] = 0
           
    for j in range(imfiltered_df.shape[0]):
        if imgjs_df.iloc[i, 3] == imfiltered_df.iloc[j, 2]:
            imgjs_df.iloc[i, 2] = imfiltered_df.iloc[j, 35]
            break

In [44]:
imgjs_df.head(100)

,ADMIN,year,value,ISO_A3,geometry
0,Aruba,2019,0.000000,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,2019,81.640100,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,2019,4.098920,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,2019,0.675941,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,2019,14.011200,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."
...,...,...,...,...,...
95,Guyana,2019,2.431930,GUY,"MULTIPOLYGON (((-58.44103 6.90477, -58.46052 6..."
96,Hong Kong S.A.R.,2019,4120.910000,HKG,"MULTIPOLYGON (((114.11744 22.22187, 114.12574 ..."
97,Heard Island and McDonald Islands,2019,0.000000,HMD,"POLYGON ((73.73512 -53.11240, 73.76189 -53.118..."
98,Honduras,2019,74.189300,HND,"MULTIPOLYGON (((-87.62434 13.25292, -87.64656 ..."


In [46]:
exgjs_df.head()

,ADMIN,year,value,ISO_A3,geometry
0,Aruba,2019,0,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,2019,8888,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,2019,8888,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,2019,8888,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,2019,8888,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


In [55]:
#Populate exgjs_df with 2019 export values
#remove the blank space from iso_a3, countries columns

for k in range(exfiltered_df.shape[0]):
        exfiltered_df.iloc[k, 2] = exfiltered_df.iloc[k, 2].strip()
        exfiltered_df.iloc[k, 1] = exfiltered_df.iloc[k, 1].strip()
        
for i in range(exgjs_df.shape[0]):
    exgjs_df.iloc[i, 2] = 0
           
    for j in range(exfiltered_df.shape[0]):
        if exgjs_df.iloc[i, 3] == exfiltered_df.iloc[j, 2]:
            exgjs_df.iloc[i, 2] = exfiltered_df.iloc[j, 35]
            break

In [56]:
exgjs_df.head(100)

,ADMIN,year,value,ISO_A3,geometry
0,Aruba,2019,0.00000,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,2019,9.90355,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,2019,6.46690,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,2019,0.00000,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,2019,5.25364,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."
...,...,...,...,...,...
95,Guyana,2019,9.44750,GUY,"MULTIPOLYGON (((-58.44103 6.90477, -58.46052 6..."
96,Hong Kong S.A.R.,2019,9725.03000,HKG,"MULTIPOLYGON (((114.11744 22.22187, 114.12574 ..."
97,Heard Island and McDonald Islands,2019,0.00000,HMD,"POLYGON ((73.73512 -53.11240, 73.76189 -53.118..."
98,Honduras,2019,3.81829,HND,"MULTIPOLYGON (((-87.62434 13.25292, -87.64656 ..."


In [57]:
print([x for x in imfiltered_df.iloc[:, 1]])

['China', 'United States', 'Japan', 'Germany', 'Thailand', 'United Kingdom', 'Singapore', 'New Zealand', 'Republic of Korea', 'Malaysia', 'Italy (d)', 'Indonesia', 'France (c)', 'Vietnam', 'India', 'Taiwan', 'United Arab Emirates', 'Canada', 'Papua New Guinea', 'Switzerland (f)', 'Netherlands', 'Hong Kong (SAR of China)', 'Ireland', 'Mexico', 'Spain', 'Sweden', 'Belgium (b)', 'Austria', 'Philippines', 'South Africa', 'Fiji', 'Denmark', 'Turkey', 'Finland', 'Poland', 'Bangladesh', 'Brunei', 'Israel', 'Brazil', 'Greece', 'Norway', 'Algeria', 'Argentina', 'Libya', 'Chile', 'Czech Republic', 'Hungary', 'Sri Lanka', 'Saudi Arabia', 'Nigeria', 'Russian Federation (e)', 'Pakistan', 'Cambodia', 'Slovak Republic', 'Portugal', 'Peru', 'Lebanon', 'Puerto Rico', 'Qatar', 'Estonia', 'Bermuda', 'Cyprus', 'Vanuatu', 'Romania', 'Egypt', 'Colombia', 'Luxembourg', 'Mauritius', 'Kuwait', 'Solomon Islands', 'Slovenia', 'Republic of Congo', 'Nepal', 'Timor-Leste', 'Kenya', 'Iran', 'Costa Rica', 'Malta', 'C

In [137]:
gjs_df.head()

,ADMIN,year,value,ISO_A3,geometry
0,Aruba,2019,8888,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,2019,8888,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,2019,8888,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,2019,8888,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,2019,8888,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


In [58]:
imgjs_df.to_file("csv&geojson/mycountries2019im.geojson", driver="GeoJSON")
exgjs_df.to_file("csv&geojson/mycountries2019ex.geojson", driver="GeoJSON")

In [65]:
ecolumn_list = [c for c in exfiltered_df.columns]
ecol_use_list = ecolumn_list[3:]
print(ecol_use_list)

['y_1987', 'y_1988', 'y_1989', 'y_1990', 'y_1991', 'y_1992', 'y_1993', 'y_1994', 'y_1995', 'y_1996', 'y_1997', 'y_1998', 'y_1999', 'y_2000', 'y_2001', 'y_2002', 'y_2003', 'y_2004', 'y_2005', 'y_2006', 'y_2007', 'y_2008', 'y_2009', 'y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016', 'y_2017', 'y_2018', 'y_2019']


In [2]:
#To build MongoDB and store the DATA
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.project2DB

In [16]:
exports = db.exports
imports = db.imports
top25exports = db.top25exports
top25imports = db.top25imports

In [17]:
completeexports = db.completeexports
completeimports = db.completeimports
imgjs = db.imgjs
exgjs = db.exgjs

In [21]:
# db.exports.delete_many({})
# db.imports.delete_many({})
# db.top25exports.delete_many({})
# db.top25imports.delete_many({})
# db.completeexports.delete_many({})
# db.completeimports.delete_many({})
# db.imgjs.delete_many({})
# db.imgjs.delete_many({})

In [63]:
print(len(exfiltered_df["countries"]))

215


In [76]:
# build json file for db.completeexports

for i in ecol_use_list:
     
    temp_list =[]

    temp_df = exfiltered_df[["countries", "iso_a3", i ]].copy()
    
    
#     print(stcutted_df.head())
    
    for j in range(0, len(temp_df["countries"])):
        temp_dict = {}
        temp_dict["country"] = temp_df.iloc[j, 0]
        temp_dict["iso_a3"] = temp_df.iloc[j, 1]
        temp_dict["value"] = temp_df.iloc[j, 2]
        
        
        temp_list.append(temp_dict)
             
    temp_df.empty    
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.completeexports.insert_one(bag)
       

In [77]:
for i in ecol_use_list:
     
    temp_list =[]

    temp_df = imfiltered_df[["countries","iso_a3", i ]].copy()
    
    
#     print(stcutted_df.head())
    
    for j in range(0, len(temp_df["countries"])):
        temp_dict = {}
        temp_dict["country"] = temp_df.iloc[j, 0]
        temp_dict["iso_a3"] = temp_df.iloc[j, 1]
        temp_dict["value"] = temp_df.iloc[j, 2]
    
        temp_list.append(temp_dict)
             
    temp_df.empty    
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.completeimports.insert_one(bag)
       

In [91]:
# build json file for db.exports

for i in ecol_use_list:
     
    temp_list =[]

    temp_df = exfiltered_df[["countries","iso_a3", i ]].copy()
    sorted_df = temp_df.sort_values(by=[i], ascending=False)
    stcutted_df = sorted_df.iloc[:30, :]
                                
#     print(stcutted_df.head())
    
    for j in range(0, 30):
        temp_dict = {}
        temp_dict["country"] = stcutted_df.iloc[j, 0] 
        temp_dict["iso_a3"] = stcutted_df.iloc[j, 1]
        temp_dict["value"] = stcutted_df.iloc[j, 2]
    
        temp_list.append(temp_dict)
             
    temp_df.empty
    sorted_df.empty
    stcutted_df.empty
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.exports.insert_one(bag)
    
    

In [92]:
# build json file for db.imports

for i in ecol_use_list:
     
    temp_list =[]

    temp_df = imfiltered_df[["countries", "iso_a3", i ]].copy()
    sorted_df = temp_df.sort_values(by=[i], ascending=False)
    stcutted_df = sorted_df.iloc[:30, :]
    
#     print(stcutted_df.head())
    
    for j in range(0, 30):
        temp_dict = {}
        temp_dict["country"] = stcutted_df.iloc[j, 0]
        temp_dict["iso_a3"] = stcutted_df.iloc[j, 1]
        temp_dict["value"] = stcutted_df.iloc[j, 2]
    
        temp_list.append(temp_dict)
             
    temp_df.empty
    sorted_df.empty
    stcutted_df.empty
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.imports.insert_one(bag)  

In [12]:
# importtop25_df columns' name list
it25column_list = [c for c in importtop25_df.columns]
it25col_use_list = it25column_list[2:]
print(it25col_use_list)

['y_2015', 'y_2016', 'y_2017', 'y_2018', 'y_2019']


In [15]:
print(importtop25_df.shape[0])

25


In [16]:
for k in range(importtop25_df.shape[0]):
        importtop25_df.iloc[k, 1] = importtop25_df.iloc[k, 1].strip()
        exporttop25_df.iloc[k, 1] = exporttop25_df.iloc[k, 1].strip()

In [23]:
# build json file for db.top25imports

for i in it25col_use_list:
     
    temp_list =[]

    temp_df = importtop25_df[["commodities", i ]].copy()
    sorted_df = temp_df.sort_values(by=[i], ascending=False)
#     stcutted_df = sorted_df.iloc[:25, :]
    
#     print(stcutted_df.head())
    
    for j in range(0, 25):
        temp_dict = {}
        temp_dict["commodities"] = sorted_df.iloc[j, 0] 
        temp_dict["value"] = sorted_df.iloc[j, 1]
    
        temp_list.append(temp_dict)
             
    temp_df.empty
    sorted_df.empty
    #stcutted_df.empty
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.top25imports.insert_one(bag)  

In [25]:
# exporttop25_df columns' name list
et25column_list = [c for c in importtop25_df.columns]
et25col_use_list = et25column_list[2:]
print(et25col_use_list)

['y_2015', 'y_2016', 'y_2017', 'y_2018', 'y_2019']


In [26]:
# build json file for db.top25exports

for i in et25col_use_list:
     
    temp_list =[]

    temp_df = exporttop25_df[["commodities", i ]].copy()
    sorted_df = temp_df.sort_values(by=[i], ascending=False)
#     stcutted_df = sorted_df.iloc[:25, :]
    
#     print(stcutted_df.head())
    
    for j in range(0, 25):
        temp_dict = {}
        temp_dict["commodities"] = sorted_df.iloc[j, 0] 
        temp_dict["value"] = sorted_df.iloc[j, 1]
    
        temp_list.append(temp_dict)
             
    temp_df.empty
    sorted_df.empty
#     stcutted_df.empty
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.top25exports.insert_one(bag)  

In [95]:
n = db.comleteimports.count_documents({}) 
print (n)       

33


In [3]:
exports = db.top25imports.find()
for i in exports:
    print(i)

{'_id': ObjectId('6083c8694d25fb256dafaffc'), 'y_2015': [{'commodities': 'Personal travel (excl education) services', 'value': 36426.0}, {'commodities': 'Passenger motor vehicles', 'value': 21541.778}, {'commodities': 'Refined petroleum', 'value': 16227.992}, {'commodities': 'Telecom equipment & parts', 'value': 11781.035}, {'commodities': 'Freight services', 'value': 9866.0}, {'commodities': 'Computers', 'value': 7819.856}, {'commodities': 'Medicaments (incl veterinary)', 'value': 7711.649}, {'commodities': 'Crude petroleum', 'value': 7624.6}, {'commodities': 'Goods vehicles', 'value': 7353.034}, {'commodities': 'Passenger transport services (c)', 'value': 6453.0}, {'commodities': 'Gold', 'value': 6044.777}, {'commodities': 'Technical & other business services', 'value': 6018.0}, {'commodities': 'Professional services', 'value': 6008.0}, {'commodities': 'Charges for intellectual property', 'value': 4760.0}, {'commodities': 'Furniture, mattresses & cushions', 'value': 4455.963}, {'comm